In [1]:
# Importy
import numpy as np
from pathlib import Path
import random
import os

# Dynamické zjištění kořenového adresáře projektu
try:
    current_script_path = Path(__file__).resolve()
    project_root = current_script_path.parent.parent 
except NameError:
    cwd = Path(os.getcwd())
    if cwd.name == "notebooks" and (cwd.parent / "data").is_dir() and (cwd.parent / "scripts").is_dir():
        project_root = cwd.parent
    elif (cwd / "data").is_dir() and (cwd / "scripts").is_dir():
        project_root = cwd
    else:
        # Pokud struktura není rozpoznána
        project_root = Path(".").resolve()
        print(f"Warning: Could not reliably determine project root. Using CWD: {project_root}")

print(f"Project root determined as: {project_root}")

# Importy modulů ze složky subcipher
from subcipher.mh import crack
from subcipher.io import export_result
from subcipher.codec import encrypt, decrypt
from subcipher.text_utils import clean_text
from subcipher.alphabet import ALPHABET

print("Subcipher modules imported successfully.")

# --- Definice cest k souborům do rootu ---

# Cesta k referenční matici vytvořené z krakatit.txt
model_path = project_root / "data" / "model" / "reference_tm.npy"
print(f"Path to reference model set to: {model_path}")

# Cesta k šifrovanému textu
ciphertext_file_path = project_root / "data" / "test" / "text_1000_sample_1_ciphertext.txt" 
print(f"Path to ciphertext file set to: {ciphertext_file_path}")

# Adresář pro export výsledků (bude vytvořen, pokud neexistuje)
output_dest_dir_attack = project_root / "exports" / "notebook_attack_results"
print(f"Output directory for attack results set to: {output_dest_dir_attack}")

Project root determined as: c:\Git\DeShiftr
Subcipher modules imported successfully.
Path to reference model set to: c:\Git\DeShiftr\data\model\reference_tm.npy
Path to ciphertext file set to: c:\Git\DeShiftr\data\test\text_1000_sample_1_ciphertext.txt
Output directory for attack results set to: c:\Git\DeShiftr\exports\notebook_attack_results


In [2]:
# Načteme transition matrix
if not model_path.exists():
    print(f"Model file not found: {model_path}")
    print("Please ensure you have built and saved the reference_tm.npy first (e.g., using 02_bigram_model.ipynb or build_bigram_model.py).")
    raise FileNotFoundError(f"Model file not found: {model_path}. Please generate it first.")
else:
    tm_ref = np.load(model_path)
    print(f"Successfully loaded reference transition matrix from '{model_path}'. Shape: {tm_ref.shape}")

Successfully loaded reference transition matrix from 'c:\Git\DeShiftr\data\model\reference_tm.npy'. Shape: (27, 27)


In [3]:
# Příprava Ciphertextu

# Inicializace proměnných pro demo text, pokud by hlavní soubor nebyl nalezen
true_plaintext_demo = None
true_key_demo = None

if ciphertext_file_path.exists() and ciphertext_file_path.is_file():
    with open(ciphertext_file_path, 'r', encoding='utf-8') as f:
        raw_ciphertext = f.read()
    print(f"Loaded ciphertext from file: {ciphertext_file_path}. Length: {len(raw_ciphertext)}")
else:
    print(f"Ciphertext file {ciphertext_file_path} not found or is not a file. Using a demo ciphertext.")
    true_plaintext_demo = "TOTO_JE_DEMONSTRACNI_TEXT_KTERY_BUDE_ZASIFROVAN_A_POTOM_PROLOMEN_POMOCI_METROPOLIS_HASTINGS_ALGORITMU_SNAD_TO_VYJDE_DOBRE"
    key_list_demo = list(ALPHABET)
    random.seed(11200)
    random.shuffle(key_list_demo)
    true_key_demo = "".join(key_list_demo)
    raw_ciphertext = encrypt(true_plaintext_demo, true_key_demo)
    print(f"Using demo ciphertext. True key for demo: {true_key_demo}")
    print(f"Demo Plaintext: {true_plaintext_demo}")
    print(f"Demo Ciphertext (raw): {raw_ciphertext[:100]}...")


# Vyčíštění ciphertextu
ciphertext_to_crack = clean_text(raw_ciphertext)
print(f"Ciphertext to crack (cleaned, length {len(ciphertext_to_crack)})")

Loaded ciphertext from file: c:\Git\DeShiftr\data\test\text_1000_sample_1_ciphertext.txt. Length: 1000
Ciphertext to crack (cleaned, length 1000)


In [4]:
# Crack kryptoanalýza
iters = 20000  # Počet iterací
temp = 1.0
seed = 112000  # Nastavení seedu, pro mě nejlepší výsledek s tímto seedem / případně dáme seed = None pro random

print(f"\nStarting cryptanalysis with {iters} iterations, temp={temp}, seed={seed}")
found_key, decrypted_plaintext, best_ll = crack(
    ciphertext_to_crack,
    tm_ref,
    iters=iters,
    temp=temp,
    seed=seed
)

print(f"\nCryptanalysis finished.")
print(f"Best Log-Likelihood: {best_ll:.4f}")
print(f"Found Key:           {found_key}")
print(f"\nDecrypted Plaintext (first 200 characters):")
print(decrypted_plaintext[:200])

# Kontrola pro demo text, pokud byl použit
if true_plaintext_demo: 
    matches = sum(p == t for p, t in zip(decrypted_plaintext, true_plaintext_demo))
    accuracy = matches / len(true_plaintext_demo) if len(true_plaintext_demo) > 0 else 0
    print(f"\n--- Demo Ciphertext Check ---")
    print(f"True Key for Demo:    {true_key_demo}")
    print(f"Accuracy with true demo plaintext: {accuracy:.2%}")
    if found_key == true_key_demo:
        print("Successfully recovered the true demo key!")
    else:
        print("Did not recover the exact true demo key.")


Starting cryptanalysis with 20000 iterations, temp=1.0, seed=112000

Cryptanalysis finished.
Best Log-Likelihood: -10811.5508
Found Key:           VLZODFTHUXWSERMCJKNYIBQGP_A

Decrypted Plaintext (first 200 characters):
_VOZEM_DO_NEHO_A_ZAS_MNE_BEZI_DO_CESTY__ZACHVELA_SE_TAK_KUDY_VPRAVO_NEBO_VLEVO_TEDY_JE_KONEC_PTAL_SE_TISE_POKYVLA_HLAVOU_TEDY_JE_KONEC_OTEVREL_DVIRKA_VYSKOCIL_Z_VOZU_A_POSTAVIL_SE_PRED_KOLA_JED_REKL_C


In [5]:
# Export výsledků

if 'project_root' not in locals() or not project_root:
    print("Chyba: project_root není definován. Spusťte prosím nejprve Buňku 1.")


if 'output_dest_dir_for_attack' not in locals() and 'project_root' in locals():
     output_dest_dir_for_attack = project_root / "exports" / "notebook_attack_results"

# Určení délky a ID pro název souboru
if ciphertext_file_path.exists() and ciphertext_file_path.is_file() and "text_1000_sample_1" in str(ciphertext_file_path.name):
    output_length = 1000
    output_sample_id = 1 
else: 
    output_length = len(ciphertext_to_crack) 
    output_sample_id = 99 

print(f"\nExporting results to directory: {output_dest_dir_for_attack}")

# Vytvoření adresáře, pokud neexistuje
output_dest_dir_for_attack.mkdir(parents=True, exist_ok=True)

export_result(
    plaintext=decrypted_plaintext,
    key=found_key,
    length=output_length, 
    sample_id=output_sample_id,
    dest=output_dest_dir_for_attack
)

print("\nResults exported successfully. Check the following files:")
exported_plaintext_file = output_dest_dir_for_attack / f'text_{output_length}_sample_{output_sample_id}_plaintext.txt'
exported_key_file = output_dest_dir_for_attack / f'text_{output_length}_sample_{output_sample_id}_key.txt'

print(f"Plaintext: {exported_plaintext_file}")
print(f"Key:       {exported_key_file}")


Exporting results to directory: c:\Git\DeShiftr\exports\notebook_attack_results

Results exported successfully. Check the following files:
Plaintext: c:\Git\DeShiftr\exports\notebook_attack_results\text_1000_sample_1_plaintext.txt
Key:       c:\Git\DeShiftr\exports\notebook_attack_results\text_1000_sample_1_key.txt
